#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 4, 9)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

In [3]:
format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

In [4]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,16,S,SYNEX,"3,000",28.25,1,CP1S,SET100,2022-03-10


In [5]:
tmp = pd.read_csv('../data/orders-log.csv')
tmp['trans'] = tmp.apply(lambda row: categorise(row), axis=1)
tmp['amount'] = tmp.qty * tmp.target
tmp[tmp.active == 2].style.background_gradient(cmap = 'Blues')

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount


In [6]:
tmp.head().sort_values('spd',ascending=False).style.hide_index()

trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
S,AIMIRT,3,DOS,SET,10000,13.100000,12.800000,0.000000,0.00%,0,2022-03-14,Sell,131000.000000
B,BBL,-3,1L,SET50,2400,135.000000,136.500000,-0.500000,-0.36%,0,2022-04-22,Buy,324000.000000
B,MAKRO,-4,RD05pct,SET,1500,39.000000,40.000000,0.500000,+1.27%,0,2022-03-03,Buy,58500.000000
B,TMT,-5,ROUND,SET,15000,11.000000,11.500000,0.100000,+0.88%,0,2022-04-18,Buy,165000.000000
B,SENA,-7,RD05pct,SET,15000,4.480000,4.620000,0.020000,+0.43%,0,2022-05-10,Buy,67200.000000


### End of Tables in the process

### Print to verify before upload file

In [28]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade    name  ...)

In [29]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

In [30]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,MAKRO,"1,500",39.00,"58,500.00",RD05pct,SET,2,2022-03-03
1,B,SENA,"15,000",4.48,"67,200.00",RD05pct,SET,2,2022-05-10
2,S,AIMIRT,"10,000",13.10,"131,000.00",DOS,SET,2,2022-03-14
3,S,CPNCG,"20,000",12.50,"250,000.00",DOS,SET,2,2022-02-14
4,S,DOHOME,"2,400",22.00,"52,800.00",C21.3,SET100,2,2022-02-02
5,S,KCE,"1,000",64.00,"64,000.00",C62.75,SET50,2,2022-03-21
6,S,PTT,"3,000",39.00,"117,000.00",CP2S,SET50,2,2022-03-03
7,S,RATCH,"2,000",46.00,"92,000.00",DOS,SET50,2,2022-03-17


In [31]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B    125700.0
S    706800.0
Name: amount, dtype: float64

In [32]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,MAKRO,"1,500",39.00,"58,500.00",RD05pct,SET,2,2022-03-03
1,B,SENA,"15,000",4.48,"67,200.00",RD05pct,SET,2,2022-05-10


In [33]:
cash = 188_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(188000, 125700.0, 62300.0)

In [34]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
2,S,AIMIRT,"10,000",13.10,"131,000.00",DOS,SET,2,2022-03-14
3,S,CPNCG,"20,000",12.50,"250,000.00",DOS,SET,2,2022-02-14
4,S,DOHOME,"2,400",22.00,"52,800.00",C21.3,SET100,2,2022-02-02
5,S,KCE,"1,000",64.00,"64,000.00",C62.75,SET50,2,2022-03-21
6,S,PTT,"3,000",39.00,"117,000.00",CP2S,SET50,2,2022-03-03
7,S,RATCH,"2,000",46.00,"92,000.00",DOS,SET50,2,2022-03-17


In [35]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(188000, 706800.0, 894800.0)

### After call ord-log (must call everytime that orders change)

In [40]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [41]:
show(df[['trans','name','spd','reason','qty','target','current','active']])

trans,name,spd,reason,qty,target,current,active


In [42]:
mb = (df.trans == 'Buy') & (df.spd >= -5)
df[mb].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
0,B,BBL,-3,1L,SET50,"2,400",135.00,136.50,-0.50,-0.36%,1,2022-04-22,Buy,"324,000.00"
1,B,MAKRO,-4,RD05pct,SET,"1,500",39.00,40.00,0.50,+1.27%,2,2022-03-03,Buy,"58,500.00"
3,B,TMT,-5,ROUND,SET,"15,000",11.00,11.50,0.10,+0.88%,1,2022-04-18,Buy,"165,000.00"


In [43]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [44]:
ms = (df.trans == 'Sell') & (df.spd <= 5)
df[ms].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
4,S,AIMIRT,3,DOS,SET,"10,000",13.10,12.80,0.00,0.00%,2,2022-03-14,Sell,"131,000.00"
5,S,CPNCG,4,DOS,SET,"20,000",12.50,12.10,0.00,0.00%,2,2022-02-14,Sell,"250,000.00"
6,S,DOHOME,5,C21.3,SET100,"2,400",22.00,21.50,0.40,+1.90%,2,2022-02-02,Sell,"52,800.00"
9,S,JASIF,5,DOS,SET,"10,000",11.30,10.80,-0.10,-0.92%,1,2022-03-03,Sell,"113,000.00"
10,S,KCE,3,C62.75,SET50,"1,000",64.00,63.25,0.00,0.00%,2,2022-03-21,Sell,"64,000.00"
11,S,PTT,5,CP2S,SET50,"3,000",39.00,37.75,-0.50,-1.31%,2,2022-03-03,Sell,"117,000.00"
12,S,RATCH,5,DOS,SET50,"2,000",46.00,44.75,1.00,+2.29%,2,2022-03-17,Sell,"92,000.00"


In [45]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
0,B,BBL,-3,1L,SET50,2400,135.0,136.50,-0.5,-0.36%,1,2022-04-22,Buy,324000.0
1,B,MAKRO,-4,RD05pct,SET,1500,39.0,40.00,0.5,+1.27%,2,2022-03-03,Buy,58500.0
3,B,TMT,-5,ROUND,SET,15000,11.0,11.50,0.1,+0.88%,1,2022-04-18,Buy,165000.0
4,S,AIMIRT,3,DOS,SET,10000,13.1,12.80,0.0,0.00%,2,2022-03-14,Sell,131000.0
5,S,CPNCG,4,DOS,SET,20000,12.5,12.10,0.0,0.00%,2,2022-02-14,Sell,250000.0
6,S,DOHOME,5,C21.3,SET100,2400,22.0,21.50,0.4,+1.90%,2,2022-02-02,Sell,52800.0
9,S,JASIF,5,DOS,SET,10000,11.3,10.80,-0.1,-0.92%,1,2022-03-03,Sell,113000.0
10,S,KCE,3,C62.75,SET50,1000,64.0,63.25,0.0,0.00%,2,2022-03-21,Sell,64000.0
11,S,PTT,5,CP2S,SET50,3000,39.0,37.75,-0.5,-1.31%,2,2022-03-03,Sell,117000.0
12,S,RATCH,5,DOS,SET50,2000,46.0,44.75,1.0,+2.29%,2,2022-03-17,Sell,92000.0


In [46]:
df[mb | ms].shape[0]

10

In [47]:
mask = (df.trade == 'B')
df[mask].nlargest(5, "amount")[["name","trans","amount","spd"]].style.format(format_dict)

,name,trans,amount,spd
0,BBL,Buy,"324,000.00",-3
3,TMT,Buy,"165,000.00",-5
2,SENA,Buy,"67,200.00",-7
1,MAKRO,Buy,"58,500.00",-4


### Select source of orders between these two

In [48]:
df_out = df[mb | ms][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
0,Buy,BBL,"2,400",135.00,1,-3,136.50,-0.50,-0.36%,1L,SET50,2022-04-22
1,Buy,MAKRO,"1,500",39.00,2,-4,40.00,0.50,+1.27%,RD05pct,SET,2022-03-03
3,Buy,TMT,"15,000",11.00,1,-5,11.50,0.10,+0.88%,ROUND,SET,2022-04-18
4,Sell,AIMIRT,"10,000",13.10,2,3,12.80,0.00,0.00%,DOS,SET,2022-03-14
5,Sell,CPNCG,"20,000",12.50,2,4,12.10,0.00,0.00%,DOS,SET,2022-02-14
6,Sell,DOHOME,"2,400",22.00,2,5,21.50,0.40,+1.90%,C21.3,SET100,2022-02-02
9,Sell,JASIF,"10,000",11.30,1,5,10.80,-0.10,-0.92%,DOS,SET,2022-03-03
10,Sell,KCE,"1,000",64.00,2,3,63.25,0.00,0.00%,C62.75,SET50,2022-03-21
11,Sell,PTT,"3,000",39.00,2,5,37.75,-0.50,-1.31%,CP2S,SET50,2022-03-03
12,Sell,RATCH,"2,000",46.00,2,5,44.75,1.00,+2.29%,DOS,SET50,2022-03-17


In [49]:
df_out = df[df.active == 2][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
1,Buy,MAKRO,"1,500",39.00,2,-4,40.00,0.50,+1.27%,RD05pct,SET,2022-03-03
2,Buy,SENA,"15,000",4.48,2,-7,4.62,0.02,+0.43%,RD05pct,SET,2022-05-10
4,Sell,AIMIRT,"10,000",13.10,2,3,12.80,0.00,0.00%,DOS,SET,2022-03-14
5,Sell,CPNCG,"20,000",12.50,2,4,12.10,0.00,0.00%,DOS,SET,2022-02-14
6,Sell,DOHOME,"2,400",22.00,2,5,21.50,0.40,+1.90%,C21.3,SET100,2022-02-02
10,Sell,KCE,"1,000",64.00,2,3,63.25,0.00,0.00%,C62.75,SET50,2022-03-21
11,Sell,PTT,"3,000",39.00,2,5,37.75,-0.50,-1.31%,CP2S,SET50,2022-03-03
12,Sell,RATCH,"2,000",46.00,2,5,44.75,1.00,+2.29%,DOS,SET50,2022-03-17


In [50]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [ ]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

### Evening Process

In [ ]:
new_active = 1
old_active = 2

In [ ]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process